In [114]:
import mysql.connector
import pandas as pd
import sys

host = '54.94.131.196'
user = 'estadistica'
password = 'Estadistica2024!!'
database = 'datalake_economico'

conn = mysql.connector.connect(
    host=host, user=user, password=password, database=database
)
cursor = conn.cursor()


In [115]:
#Extraemos datos del SUPERMERCADO
query_supemercado = "SELECT * FROM supermercado_encuesta"
df_supermercado = pd.read_sql(query_supemercado,conn)
df_supermercado = df_supermercado.sort_values(by=['id_region_indec','id_provincia_indec','fecha'])

#Buscamos la fecha maxima y la fecha minima para truncar por ese rango de fechas la tabla de IPC
fecha_min = min(df_supermercado['fecha'])
fecha_max = max(df_supermercado['fecha'])

print(df_supermercado)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_820\3395479586.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_supermercado = pd.read_sql(query_supemercado,conn)


      id_region_indec  id_provincia_indec       fecha       bebidas  \
0                   1                   1  2017-01-01  3.514467e+06   
1                   1                   1  2017-02-01  3.344457e+06   
2                   1                   1  2017-03-01  3.290739e+06   
3                   1                   1  2017-04-01  3.253288e+06   
4                   1                   1  2017-05-01  2.869437e+06   
...               ...                 ...         ...           ...   
2120                7                  94  2023-09-01  7.348860e+05   
2121                7                  94  2023-10-01  8.420591e+05   
2122                7                  94  2023-11-01  9.888538e+05   
2123                7                  94  2023-12-01  1.923769e+06   
2124                7                  94  2024-01-01  1.337953e+06   

           almacen     panaderia       lacteos        carnes  \
0     6.705475e+06  9.459063e+05  2.880656e+06  2.606193e+06   
1     6.576625e+06 

In [116]:
#Extramos los datos del IPC solo hasta la fecha maxima solicitada
query_ipc = f"SELECT * FROM ipc_valores WHERE fecha BETWEEN '{fecha_min}' AND '{fecha_max}';"
df_ipc = pd.read_sql(query_ipc,conn)

#Columnas del IPC por subdivision
columnas = ['id_region_indec','general','aguasminerales_bebidas_gaseosas','bebidas_alcholicas','pan_cereales','leche_productos_lacteos_huevos','carnes_derivados','verduras_tuberculos_legumbres',
'alimentos','restaurantes_comida_fueradelhogar','aceite_grasas_mantecas','azucar_chocalate_golosina','bienes_servicios_conservacionhogar','cuidadopersonal',
'prendasdevestir_y_calzados','equipos_audivisuales_fotograficos_procesamiento_info']

#Datos del IPC por subdivision
df_total = pd.DataFrame(columns = columnas)



C:\Users\Usuario\AppData\Local\Temp\ipykernel_820\1098761186.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ipc = pd.read_sql(query_ipc,conn)


<h1> Datos del IPC por SUBDIVISION </h1>
Se usaran para la DEFLACTACION de los precios de Supermercado.

In [117]:
df_datopor_region = pd.DataFrame(columns=columnas)

regiones = [2,3,4,5,6,7]

for region in regiones:

    df_datopor_region['general'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 1) & (df_ipc['id_region'] == region)])
    df_datopor_region['aguasminerales_bebidas_gaseosas'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 13) & (df_ipc['id_region'] == region)])
    df_datopor_region['bebidas_alcholicas'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 15) & (df_ipc['id_region'] == region)])
    df_datopor_region['pan_cereales'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 4) & (df_ipc['id_region'] == region)])
    df_datopor_region['leche_productos_lacteos_huevos'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 6) & (df_ipc['id_region'] == region)])
    df_datopor_region['carnes_derivados'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 5) & (df_ipc['id_region'] == region)])
    df_datopor_region['verduras_tuberculos_legumbres'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 9) & (df_ipc['id_region'] == region)])
    df_datopor_region['alimentos'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 3) & (df_ipc['id_region'] == region)])
    df_datopor_region['restaurantes_comida_fueradelhogar'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] == 43) & (df_ipc['id_region'] == region)])
    df_datopor_region['aceite_grasas_mantecas'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] ==7) & (df_ipc['id_region'] == region)])
    df_datopor_region['azucar_chocalate_golosina'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] ==10) & (df_ipc['id_region'] == region)])
    df_datopor_region['bienes_servicios_conservacionhogar'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] ==26) & (df_ipc['id_region'] == region)])
    df_datopor_region['cuidadopersonal'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] ==45) & (df_ipc['id_region'] == region)])
    df_datopor_region['prendasdevestir_y_calzados'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] ==17) & (df_ipc['id_region'] == region)])

    #Esta subdivision solo existe en GBA, entonces se replica para cada region. id de GBA = 2 y la subdivision = 38
    df_datopor_region['equipos_audivisuales_fotograficos_procesamiento_info'] = list(df_ipc['valor'][(df_ipc['id_subdivision'] ==38) & (df_ipc['id_region'] == 2)])

    #La region se asigna ultimo ya que los las primeras filas se define el tamaño del DF por region, una vez definido asignamos el mismo valor a multiples filas
    df_datopor_region['id_region_indec'] = region

    #Concatenacion del df_total, con los datos obtenidos por region
    df_total = pd.concat([df_total,df_datopor_region])

df_total

,id_region_indec,general,aguasminerales_bebidas_gaseosas,bebidas_alcholicas,pan_cereales,leche_productos_lacteos_huevos,carnes_derivados,verduras_tuberculos_legumbres,alimentos,restaurantes_comida_fueradelhogar,aceite_grasas_mantecas,azucar_chocalate_golosina,bienes_servicios_conservacionhogar,cuidadopersonal,prendasdevestir_y_calzados,equipos_audivisuales_fotograficos_procesamiento_info
0,2,101.3130,101.4562,101.1520,101.5263,102.1541,98.9006,104.6613,101.2848,102.2730,101.6830,101.9364,100.6718,101.4831,97.8719,101.4745
1,2,103.8085,101.7053,103.1147,103.7498,105.2875,98.8632,111.8879,103.3191,104.2501,102.9812,102.9893,101.4867,103.3753,97.4253,98.7331
2,2,106.2627,104.1797,105.2341,106.6043,108.0146,104.1811,120.5272,107.1358,106.2338,104.9851,104.5350,102.4425,104.9077,102.0665,99.9789
3,2,109.0613,106.3074,106.9521,108.8026,113.7921,106.8365,121.4805,109.6145,108.6171,107.6769,105.7448,103.1592,107.0113,107.2381,100.3028
4,2,110.4607,108.0130,109.7179,109.7644,116.5728,107.9101,118.7987,110.6764,109.9565,107.5965,107.2210,108.1243,108.3694,107.9072,101.1319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,7,2280.9069,1984.6652,2095.9865,2303.1814,2597.2877,2487.0289,2729.8125,2512.7719,2423.4533,2261.7087,2912.8396,1896.5641,1910.6175,2388.4429,2509.4458
81,7,2467.3877,2197.0823,2304.3712,2560.8697,2778.2177,2609.9358,2887.6047,2693.1287,2637.3612,2472.8046,3256.8330,2040.4058,2064.3452,2633.5434,2728.5886
82,7,2762.3076,2636.2871,2598.7539,2928.5997,3069.8225,2967.6195,3326.8511,3047.5876,2943.4710,2649.7702,3590.8819,2251.5248,2323.3319,2905.2837,3027.7198
83,7,3433.7881,3762.9708,3205.5297,3858.7833,3770.5741,3870.3309,3741.2711,3861.8744,3549.1220,3524.9052,4298.5340,2804.9896,3036.4256,3390.8295,4146.3058


<h1> Agrupacion por Provincia </h1>
Vamos a buscar agrupar cada provincia con los datos del IPC que le corresponden

In [120]:
#Obtenemos en forma de LISTA los ID de la provincias
id_provincias = list(pd.unique(df_supermercado['id_provincia_indec']))

#Eliminamos NACION ya que es un dato que no usaremos. Tambien la PROV de buenos aires ya que tiene un trato particular
id_provincias.remove(1)
id_provincias.remove(6)

#Agrupamos las columnas que vamos a usar
lista_columnas = list(df_supermercado.columns)
for columnas in df_total.columns:
    lista_columnas.append(columnas)


#Df que contendra todos los datos --> FALTA: agregar las columnas que vamos a añadir
df_datos_super_deflactados = pd.DataFrame(columns=lista_columnas)
df_datos_por_provincia = pd.DataFrame(columns=lista_columnas)

for id_provincia in id_provincias:

    #Obtencion de la totalidad de los datos POR PROVINCIA
    df_datos_por_provincia[[df_supermercado.columns]] = df_supermercado[df_supermercado['id_provincia_indec'] == id_provincia]

    #Obtencion de la region
    region = pd.unique(df_datos_por_provincia['id_region_indec'])[0]

    df_datos_por_provincia['general'] = df_total['general'][df_total['id_region_indec'] == region]

    print(df_datos_por_provincia['general'])




ValueError: Columns must be same length as key